In [1]:
import json
import regex as re
import dotenv
dotenv.load_dotenv()


import pandas as pd
import sys
sys.path.insert(0, '/tinytroupe')

import tinytroupe
from tinytroupe import control
from tinytroupe.agent import TinyPerson, TinyToolUse
from tinytroupe.tools.tiny_tool import TinyTool
import tools
from tinytroupe.tools import TinyWordProcessor
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.extraction import ResultsExtractor, ResultsReducer
from typing import Dict, Any

Looking for default config on: /Users/varunvenkatesh/Documents/GitHub/tinytroupe_project/tinytroupe/tinytroupe/utils/../config.ini
Found custom config on: /Users/varunvenkatesh/Documents/GitHub/tinytroupe_project/config.ini

!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!


Current TinyTroupe configuration 
[OpenAI]
api_type = openai
azure_api_version = 2024-08-01-preview
model = gpt-4o-mini
max_tokens = 4000
temperature = 1.2
freq_penalty = 0.0
presence_penalty = 0.0
timeout = 60
max_attempts = 5
waiting_time = 2
exponential_backoff_factor = 5
embedding_model = text-embedding-3-small
cache_api_calls = False
cache_file_name = openai_api_cache.pickle
max_content_display_length = 1024
azure_embedding_model_api_version = 2023-05-15

[Simulation]
rai_harmful_content_

In [2]:

class SupplyChainOrderTool(TinyTool):

    def __init__(self, name="SupplyChainOrderTool", description="Tool to place and track part orders.", owner=None):
        super().__init__(name, description, owner)
        self.order_log = pd.read_csv("../supply_chain_requests.csv")  # Stores all placed orders

    def _process_action(self, agent, action: dict) -> bool:
        """
        Process the action dictionary.

        Expected action format:
        {
            "parts": ["A", "B"],
            "requested_by": "2025-04-10",
            "promised_by": "2025-04-11"
        }
        """
        print(action)
        if not all(k in action for k in ["order_no","parts", "requested_by", "promised_by"]):
            raise ValueError("Action must include order_no, parts, requested_by, and promised_by.")

        order = {
            "order_no": action["order_no"],
            "parts": action["parts"],
            "requested_by": action["requested_by"],
            "promised_by": action["promised_by"],
            "status": "Pending"
        }

        self.order_log.loc[order["order_no"]] = [order['order_no'], order['parts'], order['requested_by'], order['promised_by'], order['status']]
        print(f"[{self.name}] Order placed by {agent.name}: {order}")
        return True

    def actions_definitions_prompt(self) -> str:
        return (
            "You can use this tool to place supply chain part orders.\n"
            "Action format: {\"parts\": [<part_names>], \"requested_by\": <YYYY-MM-DD>, \"promised_by\": <YYYY-MM-DD>}"
        )

    def actions_constraints_prompt(self) -> str:
        return (
            "Constraints: You must include all required fields. Dates must be in YYYY-MM-DD format. "
            "You cannot order more than 10 parts in one action."
        )


'\nclass SupplyChainOrderTool(TinyTool):\n\n    def __init__(self, name="SupplyChainOrderTool", description="Tool to place and track part orders.", owner=None):\n        super().__init__(name, description, owner)\n        self.order_log = pd.read_csv("supply_chain_requests.csv")  # Stores all placed orders\n\n    def _process_action(self, agent, action: dict) -> bool:\n        """\n        Process the action dictionary.\n\n        Expected action format:\n        {\n            "parts": ["A", "B"],\n            "requested_by": "2025-04-10",\n            "promised_by": "2025-04-11"\n        }\n        """\n        print(action)\n        if not all(k in action for k in ["order_no","parts", "requested_by", "promised_by"]):\n            raise ValueError("Action must include order_no, parts, requested_by, and promised_by.")\n\n        order = {\n            "order_no": action["order_no"],\n            "parts": action["parts"],\n            "requested_by": action["requested_by"],\n          

In [3]:
def create_elena():
    elena = TinyPerson("Elena Martinez")
    elena.define("nationality", "American")
    elena.define('age', 45)
    elena.define('gender', 'Female')
    elena.define('residence', 'New York, USA')
    elena.define('behaviors', {
                        'general': [
                            'Maintains the list of orders and their statuses.',
                            'Communicates with customers about order statuses.',
                            'Updates order statuses and communicates the new order list to all customers.']})
    return elena

In [4]:
orders =    """
            Orders:
            1. Parts A, C; Requested by: 2025-04-09; Promised by: 2025-04-10; Status: Pending
            2. Parts B, D; Requested by: 2025-04-08; Promised by: 2025-04-09; Status: Pending
            3. Parts E, F; Requested by: 2025-04-07; Promised by: 2025-04-08; Status: Pending
            4. Parts G, H; Requested by: 2025-04-06; Promised by: 2025-04-07; Status: Pending
            5. Parts I, J; Requested by: 2025-04-05; Promised by: 2025-04-06; Status: Pending
            """

In [5]:
coo = create_elena()
tool_use_faculty = TinyToolUse(tools=[SupplyChainOrderTool()])
coo.add_mental_faculties([tool_use_faculty])
customer = TinyPerson("Customer")
customer.define("description", "You are a customer who has can inquire about order status and communicate any information that may impact order delivery.")

In [6]:
#world = TinyWorld(name="Supply Chain Management", agents=[coo, customer])
#world.make_everyone_accessible()
#world.broadcast(f"orders: {orders}")
coo.listen(orders)
coo.listen_and_act("Can you provide me with the details of order #3?")

USER --> Elena Martinez: [CONVERSATION] 
          >              Orders:             1. Parts A, C; Requested by: 2025-04-09; Promised by:
          > 2025-04-10; Status: Pending             2. Parts B, D; Requested by: 2025-04-08;
          > Promised by: 2025-04-09; Status: Pending             3. Parts E, F; Requested by:
          > 2025-04-07; Promised by: 2025-04-08; Status: Pending             4. Parts G, H;
          > Requested by: 2025-04-06; Promised by: 2025-04-07; Status: Pending             5. Parts
          > I, J; Requested by: 2025-04-05; Promised by: 2025-04-06; Status: Pending

USER --> Elena Martinez: [CONVERSATION] 
          > Can you provide me with the details of order #3?

Elena Martinez acts: [THINK] 
                    > I need to provide the details of order #3. The order includes Parts E and F,
                    > requested by 2025-04-07, promised by 2025-04-08, and its status is pending.

Elena Martinez acts: [TALK] 
                    > Order #3 includes Parts E and F. It was requested on 2025-04-07, promised by
                    > 2025-04-08, and its status is currently pending.

Elena Martinez acts: [DONE] 

In [7]:
coo.listen_and_act("You are late on order #3 by 5 days. Please update the status and new delivery date and communicate the new order list.")

USER --> Elena Martinez: [CONVERSATION] 
          > You are late on order #3 by 5 days. Please update the status and new delivery date and
          > communicate the new order list.

Elena Martinez acts: [THINK] 
                    > I need to update the status of order #3 since it is late by 5 days. I should
                    > also set a new delivery date and prepare to communicate the updated order list
                    > to the customers.

Elena Martinez acts: [THINK] 
                    > I need to set a new delivery date for order #3. Since it was promised by
                    > 2025-04-08 and is now late, I will update it to 2025-04-13. I also need to
                    > prepare the new order list to communicate to the customers.

Elena Martinez acts: [THINK] 
                    > I need to update the status of order #3 to reflect the new delivery date of
                    > 2025-04-13. I should also prepare to communicate this update to the customers
                    > along with the new order list.

Elena Martinez acts: [TALK] 
                    > I will update the status of order #3 to reflect the new delivery date of
                    > 2025-04-13. I will also prepare the new order list to communicate to the
                    > customers.

Elena Martinez acts: [DONE] 

In [8]:
coo.listen_and_act("Can you provide me with the updated details of order #3?")

USER --> Elena Martinez: [CONVERSATION] 
          > Can you provide me with the updated details of order #3?

Elena Martinez acts: [THINK] 
                    > I need to provide the updated details of order #3. The status is now updated
                    > to reflect a new delivery date of 2025-04-13. I should communicate this
                    > clearly to the customer.

Elena Martinez acts: [TALK] 
                    > The updated details for order #3 are as follows: The status is now updated to
                    > reflect a new delivery date of 2025-04-13. Please let me know if you need any
                    > further information.

Elena Martinez acts: [DONE] 

In [9]:
coo.listen_and_act("Can you provide me with the updated order list in its entirety. Please format the results in a table.")

USER --> Elena Martinez: [CONVERSATION] 
          > Can you provide me with the updated order list in its entirety. Please format the
          > results in a table.

Elena Martinez acts: [THINK] 
                    > I need to compile the updated order list in its entirety and format it in a
                    > table for the customer. I will include all the orders with their details.

Elena Martinez acts: [THINK] 
                    > I need to create a table for the updated order list. I will include all orders
                    > with their details: order number, parts, requested date, promised date, and
                    > status.

Elena Martinez acts: [TALK] 
                    > Here is the updated order list in a table format:  | Order Number | Parts   |
                    > Requested By | Promised By | Status  |
                    > |--------------|---------|--------------|-------------|---------| | 1
                    > | A, C   | 2025-04-09   | 2025-04-10  | Pending | | 2            | B, D   |
                    > 2025-04-08   | 2025-04-09  | Pending | | 3            | E, F   | 2025-04-07
                    > | 2025-04-13  | Updated | | 4            | G, H   | 2025-04-06   | 2025-04-07
                    > | Pending | | 5            | I, J   | 2025-04-05   | 2025-04-06  | Pending |

Elena Martinez acts: [DONE] 

In [10]:
coo.listen_and_act("You have just received an order for parts A, B, and C. The requested by and promised by dates are 2025-04-10 and 2025-04-11 respectively. Please place the order and give me the updated order log in a table format.")

USER --> Elena Martinez: [CONVERSATION] 
          > You have just received an order for parts A, B, and C. The requested by and promised by
          > dates are 2025-04-10 and 2025-04-11 respectively. Please place the order and give me the
          > updated order log in a table format.

Elena Martinez acts: [THINK] 
                    > I need to place the new order for parts A, B, and C with the requested by date
                    > of 2025-04-10 and promised by date of 2025-04-11. After placing the order, I
                    > will update the order log and present it in a table format.

Elena Martinez acts: [THINK] 
                    > I need to place the order for parts A, B, and C. I will update the order log
                    > to include this new order and then present the updated log in a table format.

Elena Martinez acts: [TALK] 
                    > I will place the order for parts A, B, and C with the requested by date of
                    > 2025-04-10 and promised by date of 2025-04-11. After that, I will update the
                    > order log accordingly.

Elena Martinez acts: [THINK] 
                    > I need to update the order log to include the new order for parts A, B, and C.
                    > I will add this order to the existing list and then format the entire order
                    > log in a table for clarity.

Elena Martinez acts: [THINK] 
                    > I need to compile the updated order log, including the new order for parts A,
                    > B, and C. I will ensure that the table reflects all orders accurately,
                    > including the new one.

Elena Martinez acts: [TALK] 
                    > Here is the updated order log in a table format:  | Order Number | Parts   |
                    > Requested By | Promised By | Status  |
                    > |--------------|---------|--------------|-------------|---------| | 1
                    > | A, C   | 2025-04-09   | 2025-04-10  | Pending | | 2            | B, D   |
                    > 2025-04-08   | 2025-04-09  | Pending | | 3            | E, F   | 2025-04-07
                    > | 2025-04-13  | Updated | | 4            | G, H   | 2025-04-06   | 2025-04-07
                    > | Pending | | 5            | I, J   | 2025-04-05   | 2025-04-06  | Pending | |
                    > 6            | A, B, C | 2025-04-10   | 2025-04-11  | New     |

Elena Martinez acts: [DONE] 